In [1]:
import os
import numpy as np
import cv2
import pandas as pd


from module.model import Model
from module.layers import Dense
from module.activations import ReLU, Softmax
from module.optimizers import Adam
from module.losses import CategoricalCrossentropy
from module.accuracy import Categorical_Accuracy
from module.utils import one_hot_encoding
from module.utils import train_test_split


In [6]:
data = pd.read_csv('data/digit-recognizer/train.csv') 
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y = data['label'].to_numpy()
X = data.drop(['label'], axis=1).to_numpy()
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size = 1000, random_state = 42, shuffle = True)

In [9]:
category = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
label_map = {cat: i for i, cat in enumerate(category)}

In [10]:
y_train = one_hot_encoding(y_train, category)
y_test = one_hot_encoding(y_test, category)

In [12]:
# Instantiate the model
model = Model()


# Add layers
model.add(Dense(X_train.shape[1], 128))
model.add(ReLU())
model.add(Dense(128, 128))
model.add(ReLU())
model.add(Dense(128, 10))
model.add(Softmax())

# Set loss, optimizer and accuracy objects
model.set(
    loss=CategoricalCrossentropy(),
    optimizer=Adam(learning_rate=1e-3, decay=1e-3),
    accuracy=Categorical_Accuracy()
)

# Finalize the model
model.finalize()

# Train the model
model.train(X_train, y_train, validation_data=(X_test, y_test),
            epochs=10, batch_size=128, print_every=100)

epoch: 1
step: 0, acc: 0.086, loss: 2.301 (data_loss: 2.301, reg_loss: 0.000), lr: 0.001
step: 100, acc: 0.867, loss: 0.332 (data_loss: 0.332, reg_loss: 0.000), lr: 0.0009090909090909091
step: 200, acc: 0.930, loss: 0.160 (data_loss: 0.160, reg_loss: 0.000), lr: 0.0008333333333333334
step: 300, acc: 0.930, loss: 0.209 (data_loss: 0.209, reg_loss: 0.000), lr: 0.0007692307692307692
step: 320, acc: 0.950, loss: 0.077 (data_loss: 0.077, reg_loss: 0.000), lr: 0.0007575757575757576
training, acc: 0.919, loss: 0.262 (data_loss: 0.262, reg_loss: 0.000), lr: 0.0007575757575757576
validation, acc: 0.963, loss: 0.133
epoch: 2
step: 0, acc: 0.969, loss: 0.123 (data_loss: 0.123, reg_loss: 0.000), lr: 0.000757002271006813
step: 100, acc: 0.945, loss: 0.195 (data_loss: 0.195, reg_loss: 0.000), lr: 0.0007037297677691766
step: 200, acc: 0.969, loss: 0.073 (data_loss: 0.073, reg_loss: 0.000), lr: 0.0006574621959237344
step: 300, acc: 0.969, loss: 0.094 (data_loss: 0.094, reg_loss: 0.000), lr: 0.00061690

In [15]:
# Predict on the image
confidences = model.predict(X_test)

# Get prediction instead of confidence levels
predictions = model.output_layer_activation.predictions(confidences)

# Get label name from label index
prediction = label_map[predictions[2]]

print(f'predicted: {prediction}')
print(f'true_value: {np.argmax(y_test[2])}')

predicted: 9
true_value: 9
